In [28]:
from random import choices, randint, randrange, random, sample, seed 
from collections import namedtuple
from typing import List, Optional, Callable, Tuple
import numpy
#Choices(, , # of draws from the population (2 , a pair)) is a function from the random module of python
# Use of tuples so that we can store /= types of data Vs array 
#Tuples are the fastest data structure 
#fix a seed for reproducitbilty later 

In [29]:
import numpy as np # importing numpy for matrix operations 
from scipy import *
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display 
import itertools
from IPython.display import display
import random, operator

In [30]:
import numpy as np # importing numpy for matrix operations 
from scipy import *
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display 
import itertools
from IPython.display import display
import random, operator

In [31]:
seed(0) #Fix seed 

In [32]:
class Firm:
    #class attributes intialisation, to be updated with the addition of each instance
    _registry = []
    
    def __init__(self, index, capacity, max_number_of_vessel, design_speed, 
                 min_speed, max_speed, main_engine_power, aux_engine_power,
                 fix_cost, fuel_type):
        '''__init__ a method to describe the poperty that all shipping firm have'''
        self._registry.append(self)
        self.index = index
        self.capacity = capacity # Vessel capacity in TEU per firm, 
        self.max_number_of_vessel = max_number_of_vessel # # of vessels per firms
        self.design_speed = design_speed  #design speed of the vessel in knots
        self.min_speed = min_speed #min vessel speed
        self.max_speed = max_speed #max vessel speed         
        self.main_engine_power =  main_engine_power  # PS_m : main engine power in kW
        self.aux_engine_power =  aux_engine_power  # auxiliary engine power [kW] 
        self.fix_cost = fix_cost #Daily cost of vessel (USD/Day) $25,000
        self.fuel_type = fuel_type
        #self.bau_emission = bau_emission
         
   
    # Firm methods 
    def get_market_share(self , market_instance):
        '''return market share of the firm accoding to it s capacity'''
        self.transport_capacity = np.multiply(self.max_number_of_vessel , self.capacity)
        self.market_share= np.divide(self.transport_capacity, market_instance.market_capacity)
        return self.market_share
        
    def get_firm_param(self, market_instance):
        self.psy = np.multiply (self.market_share , market_instance.market_psy)
        return self.psy
    
    def get_firm_demand(self, market_instance):
        self.firm_demand = np.multiply (self.market_share , market_instance.market_demand)
        return self.firm_demand 
    
    
    def get_min_number_of_vessel(self, market_instance, operational_speed):
        self.time_at_sea = np.true_divide(market_instance.distance,  operational_speed)
        self.voyage_time = self.time_at_sea  + market_instance.port_time 
        self.number_of_trips_to_meet_demand = np.true_divide(self.firm_demand[market_instance.period_index], self.capacity)
        self.max_trips_per_vessel = np.true_divide(market_instance.annual_working_time, self.voyage_time)
        self.min_number_of_vessel_to_meet_demand = np.true_divide(self.number_of_trips_to_meet_demand, self.max_trips_per_vessel)
        return self.min_number_of_vessel_to_meet_demand, self.number_of_trips_to_meet_demand, self.time_at_sea 
    
    def get_number_of_vessel(self):
        self.number_of_vessel = np.ceil(self.min_number_of_vessel_to_meet_demand)
        return self.number_of_vessel
        
        
    def get_ship_energy_efficiency(self, market_instance):
        self.main_fuel_parameter = market_instance.SFOC_main * market_instance.eng_load_main * self.main_engine_power * 10**(-6)
        self.ship_energy = np.multiply(self.main_fuel_parameter, np.power(self.design_speed, -3) )      
        return self.ship_energy
        
    def get_main_fuel_cons(self, market_instance, operational_speed):
        self.main_fuel_cons = market_instance.distance * self.ship_energy * operational_speed**2 *self.number_of_trips_to_meet_demand
        return self.main_fuel_cons
        
    def get_aux_fuel_cons(self, market_instance): 
        self.aux_fuel_parameter = market_instance.SFOC_aux * market_instance.eng_load_aux * self.aux_engine_power * 10**(-6)
        self.aux_fuel_cons =  self.aux_fuel_parameter * self.time_at_sea 
        return self.aux_fuel_cons 
        
    def get_fuel_cost(self, market_instance):
        self.aux_fuel_cost = self.aux_fuel_cons  * market_instance.fuel_data['MGO']['price'] 
        self.main_fuel_cost = self.main_fuel_cons * market_instance.fuel_data[self.fuel_type]['price']  
        self.fuel_cost = self.main_fuel_cost + self.aux_fuel_cost
        return self.fuel_cost
        
    
    def get_total_cost(self):
        self.operating_cost = self.fix_cost * self.number_of_vessel #Fixed Cost
        self.total_cost = self.operating_cost + self.fuel_cost
        return self.total_cost 
    
    def get_revenue(self,market_instance):
        self.revenue = self.firm_demand[market_instance.period_index] * market_instance.freight_rate
        return self.revenue
    
    def get_period_profits(self):
        self.period_profit = self.revenue - self.total_cost 
        return self.period_profit
    
    def get_discounted_period_profit(self,market_instance):
        self.discounted_period_profit = (np.power(1+ market_instance.discount_rate, - market_instance.period_index),self.period_profit  )
        return self.discounted_period_profit
    
    def get_firm_carbon_emission(self, market_instance):
        self.carbon_aux_emision_factor = market_instance.fuel_data['MGO']['carbon_factor']
        self_carbon_aux_emissions = self.carbon_aux_emision_factor * self.aux_fuel_cons 

        self.carbon_main_emision_factor =  market_instance.fuel_data[self.fuel_type]['carbon_factor']
        self_carbon_main_emissions =  self.carbon_main_emision_factor * self.main_fuel_cons

        self_carbon_emissions = self_carbon_main_emissions  + self_carbon_aux_emissions
        return self_carbon_emissions
    
    def get_firm_sulfur_emission(self, market_instance):
        self.sulfur_aux_emision_factor = market_instance.fuel_data['MGO']['sulfur_factor']
        self_sulfur_aux_emissions = self.sulfur_aux_emision_factor * self.aux_fuel_cons 

        self.sulfur_main_emision_factor =  market_instance.fuel_data[self.fuel_type]['sulfur_factor']
        self_sulfur_main_emissions =  self.sulfur_main_emision_factor * self.main_fuel_cons

        self_sulfur_emissions = self_sulfur_main_emissions  + self_sulfur_aux_emissions
        return self_sulfur_emissions
class MarketConfig: 
    """Market Configuration and simulation game set up
    .. version: : 0.1
    
    Parameters
    ----------
    
    Attributes
    ----------
    
    """
    #class attributes
    t0 = 2016 #The starting year for evaluation of the pay-offs
    T = 35  # Planning Horizon t = 2016,...,2040 # 100 year scope 
    year = np.arange(2016, 2051)
    distance = 11810 #nautical miles #Notteboom (2006)  
    port_time = 264 #hours ==> 11 days in a year  #Notteboom (2006)
    demand_16_20 = np.array([1303780  for j in range(5)]) #in TEU #placehplder to be calibrated with the chosen route 
    annual_working_time = 6480 #hours per year, assumption   #hours per year, assumption 
    initial_freight_rate = 1800 ## in US$/TEU 
    SFOC_main = 206 #g/kWh, specific daily main engine fuel oil consumption rate
    SFOC_aux = 221 # specific fuel oil consumption of the auxiliary engine [g/kW h], 
    eng_load_main = 0.8 # % 
    eng_load_aux = 0.5 # engine load of the auxiliary engine [\%]


    def __init__(self, number_of_firms, demand_income_elasticity, demand_price_elasticity, 
                 freight_rate, fuel_data, discount_rate ):
        '''__init__ a method to describe the poperty that the shipping market and simulation game has'''
        self.number_of_firms = number_of_firms
        self.demand_income_elasticity = demand_income_elasticity #Constant income elasticity #IMF
        self.demand_price_elasticity =  demand_price_elasticity #Constant own price elasticity#IMF
        self.freight_rate = freight_rate ## in US$/TEU
        self.fuel_data = fuel_data 
        self.discount_rate = discount_rate 
        #self.bau_industry_emission =  bau_industry_emission 
        #self.bau_pollution_stock = bau_pollution_stock
        self.market_capacity = 0 
        self.pollution_stock = 0
        self.industry_damage = 0
        self.global_abatement_benefits = 0
        
    
    def get_market_capacity (self, firm_instance):
        self.market_capacity += firm_instance.max_number_of_vessel * firm_instance.capacity
        return self.market_capacity 
    
    def get_freight_rate_ratio(self): 
        self.beta = np.divide(self.freight_rate, MarketConfig.initial_freight_rate) 
        return self.beta
   
    def get_market_demand(self):
        '''compute market level demand'''
        #---------> 1.Import real GDP growth data & compute GDP ratio : Source IMF@2020 #
        G_df = pd.read_csv('./data/real_growth_rate.csv') #import IMF data
        #-------> 2.Construct GDP projection path 2016-2050 based on projection growth data 
        gdp_growth= G_df.values[:,1:] 
        g =1 +(gdp_growth/100)  
        self.gdp = np.array([100.00 for j in range(MarketConfig.T)])
        for foo in range (1,MarketConfig.T):
            self.gdp[foo] = g[:,foo]* self.gdp[foo-1]
        #print(self.gdp)
        #--------->  "compute GDP ratio based on IMF@2020"
        gdp_ratio = np.array([1.00 for j in range(MarketConfig.T)])
        for moo in range (1,MarketConfig.T):
            sub_g = g[:,0:moo+1]
            #print(sub_g)
            gdp_ratio[moo] = np.prod(sub_g)
        
        #--------->  "compute freight rate ratio"
        self.freight_rate_ratio_multiplied = np.power(self.beta, self.demand_price_elasticity)
        #print(freight_rate_ratio_multiplied)
        
        #---------> "Project Transport Demand (industry demand)"
        loo =np.multiply(np.power(gdp_ratio, self.demand_income_elasticity), self.freight_rate_ratio_multiplied ) #Will need to be updated in case fuel prices are varied over time  
        self.market_demand = np.multiply(loo, MarketConfig.demand_16_20[0])
        #print(Y) # size = 1 dimesion array with size T=35 years
        self.market_psy = np.multiply(np.power(gdp_ratio, self.demand_income_elasticity), MarketConfig.demand_16_20[0] )
        return self.market_demand, self.gdp, self.market_psy
    
    def set_period_index (self, period):
        self.period_index = period
        return self.period_index

        
    def get_industry_emission(self):
        
        pass
    
    def get_pollution_stock(self):
        pass
    
    def get_damage(self):
        pass
    
    def get_global_abatement_benefits(self):
        pass 
    
    def get_firm_abatement_benefits():
        pass

In [33]:
###########################################################################
#===================> 0.Chromosome and GA parameter intialisation #==============================>
##########################################################################
#1. Initilize Chromosome, ie, candidate solutions placeholder list and specify type
Chromo = List[float] #list of 1 and 0 ?? ; need to update !! #binary encoding,..., has vessel speed and number of vessels ??
def generate_chromo(lower_bound:float , upper_bound:float , length: int) -> Chromo:
    '''A function to generate a chromosome for one solution of foalt number '''
    #random sampeling between V_min, V_max 
    #which methofs to use ; uniform, ...==> Must investigate 
    return np.random.uniform(min_speed=lower_bound, max_speed=upper_bound, k=length)


##########################################################################
#========================> 2.Initial Population#====================
###########################################################################
# #random samepling 
# #Maybe use latin hyperkeep sampeling approach 
# print(__doc__)
# import numpy as np
# np.random.seed(123)
# import matplotlib.pyplot as plt
# from skopt.space import Space
# from skopt.sampler import Sobol
# from skopt.sampler import Lhs
# from skopt.sampler import Halton
# from skopt.sampler import Hammersly
# from skopt.sampler import Grid
# from scipy.spatial.distance import pdist
# lhs = Lhs(criterion="maximin", iterations=10000)
# x = lhs.generate(space.dimensions, n_samples)
# plot_searchspace(x, 'maximin LHS')
# pdist_data.append(pdist(x).flatten())
# x_label.append("maximin")


In [34]:
###########################################################################
#========================> 2.Generating Populations #==============================>
###########################################################################
#population size at least 50 
Population = List[Chromo] # list of candidate solutions generation=pop={solu1, ..., soln}={Chromosome1, ..., Chromosome} 
def generate_population(size: int, genome_length: int) -> Population: 
    '''function to generate a population by calling generate genome multiple times 
    until our population has the desired size'''
    return [generate_chromo(chromo_lower_bound, chromo_upper_bound, chromo_length) for _ in range(size)]

In [35]:
###########################################################################
#==============================> 5.Population Fitness function #==============================>
###########################################################################
def fitness(chromo: Chromo, firm: Firm, sim_game: MarketConfig, max_number_of_vessel: int) -> float:
    min_number_vessels_sim, trips_to_meet_demand_sim, time_at_sea_sim  = firm.get_min_number_of_vessel(sim_game, chromo)
    number_vessels_sim  = firm.get_number_of_vessel()
    #Insert Model Constraints Here 
    if number_vessels_sim > max_number_of_vessel:
        return 0
    else:
        main_fuel_con_sim = firm.get_main_fuel_cons(sim_game, chromo)
        aux_fuel_con_sim = firm.get_aux_fuel_cons(sim_game)
        fuel_cost_sim = firm.get_fuel_cost(sim_game)
        total_cost_sim = firm.get_total_cost()
        revenues_sim = firm.get_revenue(sim_game)
        period_profits_sim = firm.get_period_profits()
        discounted_period_profits_sim = firm.get_discounted_period_profit(sim_game)
        #We will see about the carbon and sulfir emississons 
        carbon_emissions_sim = firm.get_firm_carbon_emission(sim_game)
        sulfur_emissions_sim = firm.get_firm_sulfur_emission(sim_game)
        return  discounted_period_profits_sim


def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
    '''the fitness fn determm how good a candidate given solution (genome) is'''
    '''insert the subroutine constraint'''
    return sum([fitness_func(genome) for genome in population])
#update this selection fn with a fitness fn that takes a genome and returns a fitness value to ake the correct choice
#keep in mind genome: Genome, things: [Thing], weight_limit: int are problem specific parameter for the knapsack


In [ ]:
#Survival of the fittest  and parent selection 
#tournement selction with constraint  : randomaly select pairs and pick the fittest 
#@ the end of the selection, we result in pairs : parents:
#1) prefer a feasible solution 
#2) if both feasible, chosse better objective
#3) if both infeasibe, chose the one with the less constraint variation


 

In [19]:
###########################################################################
#========================> 1.Callable functions #==============================>
###########################################################################
#callable fn Use functions as parameters for abstracting the problem from the algo
#===============================================================================#
#callable fn Use functions as parameters for abstracting the problem from the algo
PopulateFunc = Callable[[], Population] # a population fn that takes nothing and returns new solutiobns #=Callable[[Input], Output]
FitnessFunc = Callable[[Chromo], int] # a fitness function that takes a genome and returns a fitness value to make the correct choice
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Chromo, Chromo]]  #takes a population and a fitness fn to select 2 solutions to be the parents of our next generaation solution
CrossoverFunc = Callable[[Chromo, Chromo], Tuple[Chromo, Chromo]] #takes 2 genomes and returns 2 new genomes 
MutationFunc = Callable[[Chromo], Chromo] #takes 1 genome and sometimes returns a modified one
PrinterFunc = Callable[[Population, int, FitnessFunc], None]



In [ ]:
###########################################################################
#======================>3.Single point Crossover function #=========================>
###########################################################################
#Crossover: Reproduction from 2 parents to generate 2 offsprings 
#Not Singele crossover, instead use linear crossover
#blending = linear crossover
# the child is the avg of the 2 parents
#x_c1 = 0.5 x_p1 + 0.5 x_p2
#x_c2 = 2 x_p2 - x_p1 #take the weights and bias them towards parent 2, wheras parent 2 is the fitter one def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    '''generate a new solution for the next generation'''
    '''takes 2 genomes as parameters'''
    '''returns 2 genomes as output'''
    # ====> Crosseover function onlly makes senses if 
    # Gneomes need to be the same length for this to work
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")
    
    # lengths of genome is at least 2 otherwise there would be no point to cut the genome in half
    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1) #randomly choose an index to cut the genomes in half it 
    return a[0:p] + b[p:], b[0:p] + a[p:] # the first new solution =  the first 1/2 of the 1st genome a and the rest of the second genome b
#2nd new solution: 1st part of genome b and 2nd part of genome a


In [ ]:
#mutation: randomly perturb the values
# Uniform variation: x_new = xi + (ri - 0.5) delta_i
#Normal distribution : A gaussian with mean 0 and and a chosen standard deviation   x_new = xi +N(0, sigma) 
